In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

import os
import xarray as xr
import numpy as np
import pickle
from esio import EsioData as ed
import itertools
import seaborn as sns

In [2]:
# Update your paths here
my_dir_out = '/home/disk/sipn/nicway/python/ESIO/' # The ESIO repo dir (Where the path info file get saved)
my_data_dir = '/home/disk/sipn/nicway/data/' # Path to where SIPN2 data files are stored
public_html_fig = r'/home/disk/sipn/nicway/public_html/sipn/figures/' # Where figs for website go

In [3]:
# Create an esioData
E = ed.EsioData(data_dir = my_data_dir)

In [4]:
# Add observations
obs_list = ['G10006',  'NSIDC_0051',  'NSIDC_0079',  'NSIDC_0081']
for co in obs_list:
    E.add_obs(co, grid_file='stereo_gridinfo.nc')
E.add_obs('NSIDC_extent', grid_file='N/A')    

In [5]:
# Add Models
E.add_model(mod_name='gfdlsipn', model_label='gfdl-flor', grid_file='GFDLFLOR_gridinfo.nc')
E.add_model(mod_name='piomas', model_label='piomas', grid_file='PIOMAS_gridinfo.nc')
E.add_model(mod_name='yopp', model_label='ecmwf-yopp', grid_file='yopp_land_sea_mask.nc')
# c3S Models
E.add_model(mod_name='ukmetofficesipn', model_label='ukmo-c3s', grid_file='ukmetofficesipn_land_sea_mask.nc')
E.add_model(mod_name='ecmwfsipn', model_label='ecmwf-c3s', grid_file='ecmwfsipn_land_sea_mask.nc')
# S2S download models
E.add_model(mod_name='bom', model_label='bom-s2s', grid_file='MISSING')
E.add_model(mod_name='cma', model_label='cma-s2s', grid_file='MISSING')
E.add_model(mod_name='ecmwf', model_label='ecmwf-s2s', grid_file='MISSING')
E.add_model(mod_name='hcmr', model_label='hcmr-s2s', grid_file='MISSING')
E.add_model(mod_name='isaccnr', model_label='isac-cnr-s2s', grid_file='MISSING')
E.add_model(mod_name='jma', model_label='jma-s2s', grid_file='MISSING')
E.add_model(mod_name='metreofr', model_label='météo-fr-s2s', grid_file='MISSING')
E.add_model(mod_name='ukmo', model_label='ukmo-s2s', grid_file='MISSING')
E.add_model(mod_name='eccc', model_label='eccc-s2s', grid_file='MISSING')
E.add_model(mod_name='kma', model_label='kma-s2s', grid_file='MISSING')
E.add_model(mod_name='ncep', model_label='ncep-s2s', grid_file='MISSING')
# NAVY NEMO
E.add_model(mod_name='usnavysipn', model_label='nrl', grid_file='MISSING')
E.add_model(mod_name='usnavyncep', model_label='nrl_subx', grid_file='MISSING')
# RASM ESRL
E.add_model(mod_name='rasmesrl', model_label='rasm-esrl', grid_file='MISSING')
# NECP CFSv2
E.add_model(mod_name='noaasipn', model_label='ncep-exp', grid_file='MISSING')
E.add_model(mod_name='noaasipn_ext', model_label='ncep-ext', grid_file='MISSING')
# E.add_model(mod_name='noaasipn_ext_biascorr', model_label='ncep-ext-biascorr', grid_file='MISSING')

E.add_model(mod_name='modcansipns_3', model_label='cansips-3', grid_file='MISSING')
E.add_model(mod_name='modcansipns_4', model_label='cansips-4', grid_file='MISSING')
E.add_model(mod_name='uclsipn', model_label='ucl', grid_file='mesh_mask_nemo.N3.6_ORCA1L75.nc')

E.add_model(mod_name='szapirosipn', model_label='MPAS-CESM', grid_file='MISSING')

# MME
E.add_model(mod_name='MME', model_label='MME_avg', grid_file='MISSING')

In [6]:
# Add other path info
E.fig_dir = public_html_fig 

In [9]:
# Flag if modeled ice is predicted (True) or prescribed (False)
icePredicted = {'gfdlsipn':True, 'piomas':True, 'yopp':True, 'bom':False, 'cma':False, 'ecmwf':True, 
              'hcmr':False, 'isaccnr':False, 'jma':False, 'metreofr':True, 'ukmo':True, 'eccc':False, 
              'kma':True, 'ncep':True, 'ukmetofficesipn':True, 'ecmwfsipn':True, 'usnavysipn':True,
               'usnavyncep':True, 'rasmesrl':True, 'noaasipn':True, 'noaasipn_ext':True,
               'MME':True, 'modcansipns_3':True, 'modcansipns_4':True, 'uclsipn':True, 'szapirosipn':True}
E.icePredicted = icePredicted

In [10]:
# Check all models have a predicted flag
if not (icePredicted.keys() == E.model.keys()):
    raise ValueError("Missing some predicted flags for models...")

In [11]:
# Define colors/lines etce for each model to use in plots
cmap_c = itertools.cycle(sns.color_palette("Paired", len(E.model.keys()) ))
linecycler = itertools.cycle(["-","--","-.",":","--"])
markercycler = itertools.cycle(["*","o","s","v","x"])
mod_cc = {}
mod_ls = {}
mod_mk = {}
for (i, cmod) in enumerate(E.model.keys()):
    mod_cc[cmod] = next(cmap_c)
    mod_ls[cmod] = next(linecycler)
    mod_mk[cmod] = next(markercycler)
E.model_color = mod_cc
E.model_linestyle = mod_ls
E.model_marker= mod_mk

In [12]:
# Save to ESIO dir
print("Saving to ", my_dir_out)
E.save()

Saving to  /home/disk/sipn/nicway/python/ESIO/


In [ ]:
# # Load object to test
# E = None
# E = ed.EsioData.load()